# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)





## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
# References: 
1. https://github.com/PyTorchLightning/lightning-bolts
2. https://huggingface.co/models
3. https://github.com/ildoonet/pytorch-gradual-warmup-lr
4. grad_accum:
https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/02/19/gradient-accumulation.html 
5. Regex matching (post-processing: match [UNK] back)
    https://regexr.com/3f4vo

    https://docs.python.org/3/library/re.html

    https://www.programiz.com/python-programming/regex

    https://transbiz.com.tw/
    regex-regular-expression-ga-%E6%AD%A3%E8%A6%8F%E8%A1%A8%E7%A4%BA%E5%BC%8F/

    https://blog.finxter.com/python-regex-greedy-vs-non-greedy-quantifiers/
6. Models 
(Distilbert does not support fasttokenizer, token_type_ids)
https://github.com/huggingface/transformers/issues/2702



## Download Dataset

In [ ]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 67.8MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [ ]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 3.8MB/s 
     |███████████████████▍            | 542kB 26.0MB/s eta 0:00:01

## Import Packages

In [ ]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(2929)

In [ ]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

In [ ]:
!nvidia-smi

Sat May 22 12:33:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Model and Tokenizer




 

In [ ]:
# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)
from transformers import AutoModelForQuestionAnswering, BertTokenizer, AutoModel, AlbertForMaskedLM
# bert-base-chinese  sample code 
# wptoux/albert-chinese-large-qa good
# hfl/chinese-roberta-wwm-ext good 
# hfl/chinese-roberta-wwm-ext-large good!
# hfl/chinese-electra-180g-small-discriminator nah
# yechen/question-answering-chinese nah

# ckiplab/albert-base-chinese forward got unexpected start position
# ckiplab/albert-base-chinese-pos: forward() got an unexpected keyword argument 'start_positions'

# model = AutoModelForQuestionAnswering.from_pretrained('wptoux/albert-chinese-large-qa')
# sample code only covers preproc for FAST tokenizers (other python-based tokenizer raise errors)
# tokenizer = BertTokenizerFast.from_pretrained('wptoux/albert-chinese-large-qa')


tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-roberta-wwm-ext-large')
model = AutoModelForQuestionAnswering.from_pretrained('hfl/chinese-roberta-wwm-ext-large')

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 300 #150
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 100 # 70

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx] 
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]
        
        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn 
        # (也就是不要讓他以為answer is always in the middle)

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])
            
            ###########
            choice= random.choices([1/2, 1/3, 1/4, 1/5, 2, 3, 4, 5, 6], k = 1)[0]
            ############
            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - round(self.max_paragraph_len /choice) , len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            paragraph_id = question["paragraph_id"]
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                

                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list), paragraph_id

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_batch_size = 8
train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [ ]:
# answer = '白[UNK]紀中恐[UNK]大量滅絕'
# gt = '歷史課本有教，白堊紀中恐龍大量滅絕，因此現在看不到'
# # Hello I am (\w*\s?)him.
# answer = answer.replace('[UNK]', '\S')
# print(answer)
# match = re.search(answer, gt).group(0)
# print(match)
import regex as re 

In [ ]:
def evaluate(data, output,isTest):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong: 
    '''
    start > end
    [UNK] should be matched back to the original paragraph
    '''
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
  
    paragraph_id = data[-1].item()
    for k in range(num_of_windows): # 5 in dev set 
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            if start_index > end_index:
              temp = start_index
              start_index = end_index
              end_index = temp 
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    ########## Post-processing ################
    answer = answer.replace(' ', '')
    answer = answer.replace(',','')
    answer = answer.replace('[SEP]','')
    answer = answer.replace('[CLS]','')
    answer= answer.replace('[PAD]','')
    answer = answer.strip('的')
    if '[UNK]' in answer: 
      print('Original:', answer, end = '|')
      try:
         # sometimes it can be 'MB' which are tokenized as one word but decoded to 2 words
         # but set the boundary '[UNK]' to '\S' to prevent over-matching
        if not answer.endswith('[UNK]') and not answer.startswith('[UNK]'):
          answer = answer.replace('[UNK]', '\S+?') ####### to be fixed ######## 
        if (answer.startswith('[UNK]') or answer.endswith('[UNK]')):
          answer = re.sub(r'^\[UNK\]', '@', answer) # \: escape symbol # ^ matches beginning
          answer = re.sub(r'\[UNK\]$', '@', answer) # $ matches end 
          answer = answer.replace('@', '\S') # @ is a temp word marker 
          answer = answer.replace('[UNK]', '\S+?') 
        # find the shortest match (else might return a paragraph)
        if isTest:
          matches = re.findall(answer, test_paragraphs[paragraph_id], overlapped = True) # match back to the original paragraph 
          answer = min(matches, key = len)
        else:
          matches = re.findall(answer, dev_paragraphs[paragraph_id], overlapped = True)
          answer = min(matches, key = len)
        print('Matched:', answer)
      except: 
        print('No match.') # if a match is found, return it; else pass 
    return answer 

In [ ]:
!pip install lightning-bolts

In [ ]:
import pytorch_lightning as pl

## Training

In [ ]:
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau,ExponentialLR
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate)
accum_iter = 25 # virtually, real batch_size is batch_size * accum_iter 
# warmup_steps 
scheduler = LinearWarmupCosineAnnealingLR(optimizer,
                                          warmup_start_lr = 1e-9,
                                          warmup_epochs=200, # think of it as steps 
                                          max_epochs=num_epoch*len(train_loader)) # steps 
# scheduler = StepLR(optimizer, step_size= 1000, gamma=0.1) # 1000 steps 更新一次
if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for batch_idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], 
                       token_type_ids=data[1], 
                       attention_mask=data[2], 
                       start_positions=data[3], 
                       end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1) 
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        # Gradient Accumulation for larger batch size (sample code: 16)
        loss = output.loss
        loss = loss / accum_iter 
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        step += 1
        # Grad accum: weights update condition 
        if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
            optimizer.step()
            optimizer.zero_grad()
        ##### TODO: Apply learning rate decay #####
        scheduler.step()
      
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0 

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
              
                output = model(input_ids=data[0].squeeze().to(device), token_type_ids=data[1].squeeze().to(device),
                       attention_mask=data[2].squeeze().to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, isTest = False) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()
    
# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []
model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, isTest = True))

In [ ]:
result_file = "_.csv"
with open(result_file, 'w') as f:	
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
        # all preproc is done in evaluate()
        f.write(f"{test_question['id']},{result[i]}\n")
print(f"Completed! Result is in {result_file}")

In [ ]:
from google.colab import files 
files.download(f'{result_file}')